#### 전처리 끝~

In [28]:
import pandas as pd
import numpy as np
import json
import os

data_path = './100_Solvook_handout_DB_english (2).xlsx'
all_sheet = pd.read_excel(data_path, sheet_name= None)
sheet_names = ['handout_db', 'paragraph_db', 'handout_list', 'paragraph_list1', 'handout_type', '교과서 코드' ]
handout_db = all_sheet['1_handout_db']
paragraph_db = all_sheet['2_paragraph_db']
paragraph_list = all_sheet['4_paragraph_list']
handout_type = all_sheet['5. handout_type(81 회의 이후 변경)']

handout_db = handout_db.dropna(subset=['질문', '본문'])
selected_columns = ['본문', '조건', '선지', '정답', '질문',  '분류체계 시트 > \n문제유형_01', '분류체계 시트\n문제유형_02', '분류체계 시트\n문제유형_03', '분류체계 시트 > \nskill # , method_01', '분류체계 시트 > \nskill # , method_02',
       '분류체계 시트 > \nskill # , method_03', 'unit_id', 'unit_name', 'story_name', '교과서명']
new_df = handout_db[selected_columns]
new_df = new_df.reset_index(drop=True)
new_column_names = {
    '분류체계 시트 > \n문제유형_01' : '문제유형1',
    '분류체계 시트\n문제유형_02' : '문제유형2',
    '분류체계 시트\n문제유형_03' : '문제유형3',
    '분류체계 시트 > \nskill # , method_01' : 's&m1',
    '분류체계 시트 > \nskill # , method_02' : 's&m2', 
    '분류체계 시트 > \nskill # , method_03' : 's&m3'
}
new_df = new_df.rename(columns=new_column_names)
new_df.head(1)

,본문,조건,선지,정답,질문,문제유형1,문제유형2,문제유형3,s&m1,s&m2,s&m3,unit_id,unit_name,story_name,교과서명
0,"Girl: Minho, you look tired.\nBoy: I went to b...",NaN,(A) : ________________________________________...,(A) Was there any particular reason? Did you h...,"1. 다음 글의 대화의 빈칸 (A), (B)에 들어갈 우리말을 영작하시오.",24,NaN,NaN,"{ ""skill"": [3], ""method"": [3] }",NaN,NaN,2,"Be Smart, Be Healthy","Be Smart, Be Healthy","고등 영어 (양) (자습서, 평가문제집 포함)"


In [30]:
# remove prefix numbering 
import re
def remove_non_alphanumeric_prefix(input_string):
    pattern = '[가-힣a-zA-Z]'
    match = re.search(pattern, input_string)
    if match: cleaned_string = input_string[match.start():]
    else:     cleaned_string = input_string
    return cleaned_string

for col in ['본문','질문','선지','조건','정답']:
    for idx in range(len(new_df['질문'])):
        try:
            new_df.loc[idx, col]= re.sub('___+','___', new_df.loc[idx, col]) # replace _____(long) > __(short)
        except:
            pass
new_df['질문'] = new_df['질문'].apply(remove_non_alphanumeric_prefix)
new_df.head(1)

,본문,조건,선지,정답,질문,문제유형1,문제유형2,문제유형3,s&m1,s&m2,s&m3,unit_id,unit_name,story_name,교과서명
0,"Girl: Minho, you look tired.\nBoy: I went to b...",NaN,(A) : ___\n(B) : ___,(A) Was there any particular reason? Did you h...,"다음 글의 대화의 빈칸 (A), (B)에 들어갈 우리말을 영작하시오.",24,NaN,NaN,"{ ""skill"": [3], ""method"": [3] }",NaN,NaN,2,"Be Smart, Be Healthy","Be Smart, Be Healthy","고등 영어 (양) (자습서, 평가문제집 포함)"


In [31]:
# { "skill": [3], "method": [3] } format에서 skill값과 method 값을 추출
def extract_skill_method(s_and_m):
    try:
        s_and_m_dict = json.loads(s_and_m.replace("'", "\""))
        skill = s_and_m_dict['skill'][0] if 'skill' in s_and_m_dict else None
        method = s_and_m_dict['method'][0] if 'method' in s_and_m_dict else None
        return [skill, method]
    except:
        return []
for idx in range(1,4):
    new_df['s&m'+str(idx)] = new_df['s&m'+str(idx)].apply(extract_skill_method) 
    new_df['문제유형'+str(idx)] = new_df['문제유형'+str(idx)].apply(lambda x: [int(x)] if not pd.isna(x) else [])
new_df['s&m'] = new_df['s&m1'] + new_df['s&m2'] + new_df['s&m3']
new_df['문제유형'] = new_df['문제유형1'] + new_df['문제유형2'] + new_df['문제유형3']
new_df = new_df.reset_index(drop=True)
new_df.head(1)

,본문,조건,선지,정답,질문,문제유형1,문제유형2,문제유형3,s&m1,s&m2,s&m3,unit_id,unit_name,story_name,교과서명,s&m,문제유형
0,"Girl: Minho, you look tired.\nBoy: I went to b...",NaN,(A) : ___\n(B) : ___,(A) Was there any particular reason? Did you h...,"다음 글의 대화의 빈칸 (A), (B)에 들어갈 우리말을 영작하시오.",[24],[],[],"[3, 3]",[],[],2,"Be Smart, Be Healthy","Be Smart, Be Healthy","고등 영어 (양) (자습서, 평가문제집 포함)","[3, 3]",[24]


In [40]:
# save tr/val/te split
from sklearn.model_selection import train_test_split
tr, te_0 = train_test_split(new_df, 
                            stratify=new_df['문제유형'].apply(lambda x: x[0] if x[0] != 39 else 41),
                            test_size=0.2,
                            random_state=1)
# val / te
val, te  = train_test_split(te_0,
                            stratify=te_0['문제유형'].apply(lambda x: x[0] if x[0] != 5 else 41),
                            test_size=0.5,
                            random_state=1)
print(f'train split: {tr.shape}')
print(f'validation split: {val.shape}')
print(f'test split: {te.shape}')

data_path = './data'
tr.to_csv('./solvook_handout_tr_10_31.csv', encoding='utf-8-sig')
val.to_csv('./solvook_handout_val_10_31.csv', encoding='utf-8-sig')
te.to_csv('./solvook_handout_te_10_31.csv', encoding='utf-8-sig')

train split: (14509, 17)
validation split: (1814, 17)
test split: (1814, 17)


#### 실험

In [41]:
# sampling 100 qeury for test 
import pandas as pd
new_df = pd.read_csv('./solvook_handout_te_10_31.csv')
sample_size = 100
random_sample = new_df.sample(n=sample_size, random_state=42)  
question_list = random_sample['질문'].values.tolist()
question_list

['빈칸에 들어갈 적절한 단어를 쓰시오.',
 '다음 글의 내용과 일치하지 않는 것은?',
 '다음 아래 문장을 번역하시오.',
 '글의 흐름으로 보아, 주어진 문장이 들어가기에 가장 적절한 곳을 고르시오.',
 '빈칸에 들어갈 적절한 단어를 쓰시오.',
 '빈칸에 들어갈 적절한 단어를 쓰시오.',
 '다음 글의 밑줄 친 부분 중, 어법상 틀린 것을 두 개 골라 올바르게 고치시오.',
 '다음, 어법상 틀린 것을 찾아 고치시오. [주관식]',
 '다음 글이 들어갈 가장 알맞은 위치는?',
 '주어진 글 다음에 이어질 글의 순서로 가장 적절한 것은?',
 '빈칸에 들어갈 말로 가장 적절한 것은?',
 '위 글의 (A), (B)에 들어갈 말로 적절한 것은? 173)',
 '다음 글의 밑줄 친 부분 중, 어휘의 쓰임이 적절하지 않은 것은?',
 '다음 글의 빈칸에 들어갈 말로 가장 적절한 것은?',
 '다음 주어진 문장이 들어갈 곳으로 적절한 것은? 217)',
 '윗글의 밑줄 친 ➀~➄중 중 어법상 잘못된 부분이 들어있는것은?',
 '위 글의 빈칸 ⓐ에 들어갈 말로 가장 적절한 것은?',
 '빈칸에 들어갈 적절한 단어를 쓰시오.',
 '글의 흐름으로 보아, 주어진 문장이 들어가기에 가장 적절한 곳을 고르시오.',
 '다음 글의 밑줄 친 부분 중, 어휘의 쓰임이 적절하지 않은 것은?',
 '다음 글의 밑줄 친 부분 중, 문맥상 낱말의 쓰임이 적절하지 않은 것은?',
 '다음 글을 읽고, 울음이 실제로 기분을 나아지게하기 위해서는 어떤 조건이 필요한지를, 지문에서 찾아 우리말로 쓰시오.',
 '글의 흐름으로 보아, 주어진 문장이 들어가기에 가장 적절한 곳을 고르시오.',
 '빈칸에 들어갈 적절한 단어를 쓰시오.',
 '빈칸에 들어갈 적절한 단어를 쓰시오.',
 '다음 글의 유명 인용구들의 공통된 메시지를 한 문장으로 요약 하고자 한다. 빈칸 (A), (B)에 들어갈 말로 가장 적절한 것은?',
 '빈칸에 들어갈 적절한 단어를 쓰시오.',
 '다음 글을 읽고 각 <보기

In [78]:
np.unique(tr['s&m'])

array([list([]), list([1, 2]), list([1, 2, 1, 2]), list([1, 2, 2, 13]),
       list([1, 2, 4, 10]), list([1, 2, 4, 13]), list([1, 2, 5, 8]),
       list([1, 5]), list([1, 15]), list([1, 15, 1, 2, 5, 7]),
       list([2, 5]), list([2, 5, 1, 2, 4, 10]), list([2, 5, 1, 5]),
       list([2, 5, 1, 5, 1, 2]), list([2, 12]), list([2, 13]),
       list([2, 13, 4, 4]), list([2, 13, 4, 4, 3, 1]), list([2, 15]),
       list([3, 1]), list([3, 1, 4, 9]), list([3, 1, 4, 13]),
       list([3, 3]), list([3, 3, 3, 3]), list([3, 3, 4, 9]),
       list([3, 3, 4, 10]), list([3, 3, 4, 10, 5, 7]),
       list([3, 3, 4, 13]), list([3, 3, 5, 7]), list([3, 3, 5, 8]),
       list([4, 4]), list([4, 4, 2, 15]), list([4, 4, 3, 1]),
       list([4, 4, 3, 3]), list([4, 4, 5, 7]), list([4, 6]), list([4, 9]),
       list([4, 9, 3, 3]), list([4, 9, 4, 13]), list([4, 10]),
       list([4, 10, 3, 1]), list([4, 10, 3, 3]), list([4, 11]),
       list([4, 13]), list([4, 13, 2, 12]), list([4, 13, 2, 13]),
       list([4, 13,

In [81]:
np.unique(tr['s&m'].apply(lambda x:x[1] if x != [] else 0))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 15, 16],
      dtype=int64)

In [91]:
all_sheet = pd.read_excel('./100_Solvook_handout_DB_english (2).xlsx', sheet_name= None)
handout_type = all_sheet['5. handout_type(81 회의 이후 변경)']

handout_type.columns = handout_type.iloc[0]
handout_type = handout_type[1:]
handout_type.reset_index(drop= True, inplace=True)
quiztype_dict, skill_dict, method_dict = {}, {}, {}
for idx in range(len(handout_type)):
    row = handout_type.iloc[idx]
    quiztype_dict[row['quiz type']] = row['문제 유형 (영어)']
    skill_dict[row['skill #']] = row['skill (2depth)']
    method_dict[row['method #']] = row['method (2depth) 영어']

skill_list = list(skill_dict[key] for key in sorted(skill_dict.keys()))
method_list = list(method_dict[key] for key in sorted(method_dict.keys()))
quiztype_list = list(quiztype_dict[key] for key in sorted(quiztype_dict.keys()))

In [59]:
# candidates for skill, method, reference (newly defined after 8/1)
reference = ['vocabulary:맞는,문맥에,안에서,낱말로,괄호', 
        'grammar:어법을,어휘,고르시오,적절한,어법상', 
        'expression:이용하여, 완성하시오,단어를,주어진,밑줄',
        'content:글의,가장,적절한,다음,주어진', 
        'context:내용과,것은,일치하지,않는,글의', 
        'translate the sentence:우리말로,쓰시오,찾아,밑줄,적으시오',
        'write a sentence:이용하여,완성하시오,단어를,주어진',
        'correct the underlined:고치시오,틀린,것을,어법상,올바르게',
        'find the all-true:어법을,어휘,고르시오,적절한,맞는',
        'find the matching / the mismatched:내용과,일치하지,않는,것은,글의',
        'find the main idea:요지로,가장,적절한,것은,목적으로',
        'make an analogy:것은,없는,읽고,가장,글을',
        'find the right title:제목으로,주제로,가장,적절한,것은', 
        'arrange the sequence:이어질,순서로,다음에,주어진,가장', 
        'position the sentence:문장이,들어가기에,흐름으로,보아,가장', 
        'find the right conjunction:들어갈,말로,빈칸,것은,가장', 
        'find the correct / find the incorrect:부분,것은,다음,어법상,밑줄', 
        'find the same / find the different:다른,같은,대상이,가르키는,밑줄', 
        'find the irrelevant:흐름과,전체,문장은,관계없는,글에서']

model_name = 'gpt-3.5-turbo-0613'
model16k_name = 'gpt-3.5-turbo-16k-0613'
price = 0.0015
os.makedirs('./test', exist_ok = True)

In [ ]:
# numbered candidates for skill, method
skill_numbering = ['1. vocabulary', '2. grammar', '3. expression', '4. content', '5. context']
method_numbering = ['1. translate the sentence', '2. write or find the right word', '3. write a sentence', '4. correct the underlined',
        '5. find the all-true', '6. find the matching / the mismatched', '7. find the main idea', '8. make an analogy',
        '9. find the right title', '10. arrange the sequence', '11. position the sentence', '12. find the right conjunction',
        '13. find the correct / find the incorrect', '14. correct the incorrect', '15. find the same / find the different',
        '16. find the irrelevant']

In [60]:
# calculate the number of tokens in sentence
# https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens


In [ ]:
# plot_heatmap
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
def plot_heatmap(label, preds, label_name='Skill'):
    cm = confusion_matrix(label, preds)
    unique_list = np.unique(np.array([label, preds]))
    accuracy = sum([cm[i, i] for i in range(len(unique_list))]) / len(unique_list)

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='plasma', xticklabels=unique_list, yticklabels=unique_list)
    plt.title(f"Accuracy: {accuracy:.2f}")
    plt.xlabel('Predict')
    plt.ylabel(label_name)
    plt.title(f'{label_name} Confusion Matrix')
    plt.show()
    
    print(classification_report(label, preds))

In [ ]:
q_list_test1 = []
for question in question_list:
    prompt = f"Skill은 질문텍스트를 이해해야 하는 학습내용을 의미하고, Method는 Skill을 습득하는 방법을 담은 정보이다. Skill과 Method의 분류 리스트는 다음과 같다.\n" \
             f"Skill: {skill}\nMethod: {method}\n" \
             f"다음 질문텍스트의 Skill과 Method는 각각 무엇인지 하나씩 맞춰보세요.\n" \
             f"질문 : {question}"
    q_list_test1.append(prompt)
q_list_test1[0]

"Skill은 질문텍스트를 이해해야 하는 학습내용을 의미하고, Method는 Skill을 습득하는 방법을 담은 정보이다. Skill과 Method의 분류 리스트는 다음과 같다.\nSkill: ['vocabulary', 'grammar', 'expression', 'content', 'context']\nMethod: ['translate the sentence', 'write or find the right word', 'write a sentence', 'correct the underlined', 'find the all-true', 'find the matching / the mismatched', 'find the main idea', 'make an analogy', 'find the right title', 'arrange the sequence', 'position the sentence', 'find the right conjunction', 'find the correct / find the incorrect', 'correct the incorrect', 'find the same / find the different', 'find the irrelevant']\n다음 질문텍스트의 Skill과 Method는 각각 무엇인지 하나씩 맞춰보세요.\n질문 : 다음 글을 통해 알 수 없는 것은?"

In [ ]:
res_list_test1 = []
n_tokens = 0
for query in q_list_test1:
    messages = [{'role': 'user', 'content': query}]
    if num_tokens_from_messages(messages) > 4000: # 토큰수가 넘어가면 16k모델로 돌림
        res = openai.ChatCompletion.create(model = model16k_name, messages = messages, max_tokens = 500)
        n_tokens += res['usage']['total_tokens']*2
    else:
        res = openai.ChatCompletion.create(model = model_name, messages = messages, max_tokens = 500)
        n_tokens += res['usage']['total_tokens']
    res_list_test1.append(res['choices'][0]['message']['content'])
print(f'usage : {n_tokens/1000*price:.02f}$')
res_list_test1[:2]

usage : 0.04$


['Skill: content\nMethod: find the irrelevant',
 'Skill: content\nMethod: write a sentence']

In [ ]:
res_list_test1

['Skill: content\nMethod: find the irrelevant',
 'Skill: content\nMethod: write a sentence',
 'Skill: grammar\nMethod: correct the underlined',
 'Skill: context\nMethod: arrange the sequence',
 'Skill: vocabulary\nMethod: write or find the right word\n\n질문 : 다음 글의 괄호 (A),(B),(C) 안에서 문맥상 맞는 낱말로 가장 적절한 것은?\n\n이 질문은 주어진 문맥에서 가장 알맞은 단어를 선택하는 문제로, Skill은 vocabulary(어휘력)이고, Method는 write or find the right word(적절한 단어를 쓰거나 찾는 방법)입니다.',
 'Skill: content\nMethod: find the mismatched',
 'Skill: content\nMethod: write a sentence',
 'Skill: vocabulary/grammar\nMethod: write or find the right word',
 'Skill: context\nMethod: arrange the sequence',
 'Skill: content\nMethod: arrange the sequence',
 'Skill: grammar\nMethod: correct the underlined',
 'Skill: content\nMethod: find the main idea',
 'Skill: content\nMethod: arrange the sequence\n\n질문 : 주어진 글 (A)에 이어질 내용을 순서에 맞게 배열한 것으로 가장 적절한 것은?\n\n',
 'Skill: context\nMethod: find the right word\n\n질문의 Skill은 context입니다. 문맥을 이해하고 적절한 단어를 찾아내는 것이 필요합니다. 

정답 출력 양식  
1. skill / method 뒤에 skill_name / method_name으로 답변 ex) 'Skill: content\nMethod: write a sentence'  
2. 1번 뒤에 질문이 있는 경우가 있음. 이 질문은 물어본 질문과 같을 수도 있고 다를 수도 있음 ex) 'Skill:  ... sentence 질문 : 주어진 글 (A)... 가장 적절한 것은?\n\n',  
3. 1번 뒤에 추가 설명하는 경우 ex) "Skill: ... incorrect\n\n질문의 Skill은 'grammar'이며, Method는 'find the incorrect'입니다.",  

4. 답변을 거부하는 경우 존재 ex) '질문에 대한 정보가 부족하여 Skill과 Method를 판단할 수 없습니다. 추가 정보가 있다면 알려주세요.',     
    => 원래 질문 :'다음 빈칸 (A), (B), (C)에 들어갈 말로 순서대로 바르게 짝지은 것은?'  
5. 이상한 답변을 하는 경우 존재 ex) .. . 해야 할까요? 어떤 스킬과 메소드를 사용해야 영어 실력이 향상될까요?\n\nSkill: content\nMethod: find the right method for improving English proficiency',  
    => 원래 질문 : '다음 글의 내용과 일치하는 것은?'  

=> 대부분에 경우에 skill_name / method_name이 존재(4,5번은 질문만으로 답변하기 어려운 경우 인듯 어쩔 수 없이 오답 처리)

정답 채점 방식  
1. 소문자로 변환
2. 정규표현식을 사용하여 해당 skill_name과 method_name이 존재하는지 파악
2. 추가설명을 위해 같은 name을 여러번 언급할 수 있으므로 np.unique사용 하여 각 skill, method 별로 한번씩 언급하는지 파악
3. 그 외의 경우: skill 은 6번 method는 17번(range 밖)  

In [ ]:
def pred_parse(string , pats = ['vocabulary', 'grammar', 'expression', 'content', 'context']):
    txt_preds = []
    for idx, pat in enumerate(pats):
        for match in re.finditer(pat, string):
            txt_preds.append(match.group())
    txt_preds = np.unique(txt_preds) # 중복 처리
    if len(txt_preds) == 1:         # 1개 찾은경우
        num_pred = pats.index(txt_preds[0])
    else:                           # 예외 skill label: 5
        num_pred = len(pats)
    return txt_preds, num_pred

txt_preds, num_pred = pred_parse( "Skill: grammar\nMethod: find the incorrect\n\n질문의 Skill은 'grammar'이며, Method는 'find the incorrect'입니다.".lower(), skill)
print(txt_preds, num_pred)
txt_preds, num_pred = pred_parse( "Skill: grammar\nMethod: find the incorrect\n\n질문의 Skill은 'grammar'이며, Method는 'find the incorrect'입니다.".lower(), method)
print(txt_preds, num_pred)
print(True) if 'find the incorrect' in method else False

['grammar'] 1
[] 16


False

In [ ]:
skill_preds_test1 = []
method_preds_test1 = []
for string in res_list_test1:
    _, s = pred_parse(string.lower(), skill)
    _, m = pred_parse(string.lower(), method)
    skill_preds_test1.append(s+1)
    method_preds_test1.append(m+1)
print(skill_preds_test1[0], method_preds_test1[0])
print(skill_preds_test1[1], method_preds_test1[1])
data = [row for row in zip(res_list_test1, skill_list, method_list, skill_preds_test1, method_preds_test1)]
df_test1 = pd.DataFrame(data, columns=['Response', 'label_skill', 'label_method','pred_skill', 'pred_method'])
df_test1.to_csv("./data/test/test_1.csv")

4 16
4 3


In [19]:
from sklearn.metrics import multilabel_confusion_matrix, precision_score, recall_score, f1_score, classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_multilabel_classification
from sklearn.ensemble import RandomForestClassifier

# 데이터 생성 (임의의 Multi-label 데이터 생성)
X, y = make_multilabel_classification(n_samples=1000, n_features=20, n_classes=5, n_labels=2, random_state=42)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 학습
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train, y_train)

# 모델 예측
y_pred = classifier.predict(X_test)

# MultiLabelBinarizer를 사용하여 실제값과 예측값을 이진화
mlb = MultiLabelBinarizer()
y_test_bin = mlb.fit_transform(y_test)
y_pred_bin = mlb.transform(y_pred)

# Multi-label confusion matrix 계산
conf_matrix = multilabel_confusion_matrix(y_test_bin, y_pred_bin)

# Micro, Macro, Weighted Average에 대한 Precision, Recall, F1 Score 계산
precision_micro = precision_score(y_test_bin, y_pred_bin, average='micro')
precision_macro = precision_score(y_test_bin, y_pred_bin, average='macro')
precision_weighted = precision_score(y_test_bin, y_pred_bin, average='weighted')

recall_micro = recall_score(y_test_bin, y_pred_bin, average='micro')
recall_macro = recall_score(y_test_bin, y_pred_bin, average='macro')
recall_weighted = recall_score(y_test_bin, y_pred_bin, average='weighted')

f1_micro = f1_score(y_test_bin, y_pred_bin, average='micro')
f1_macro = f1_score(y_test_bin, y_pred_bin, average='macro')
f1_weighted = f1_score(y_test_bin, y_pred_bin, average='weighted')

# Classification Report 출력
report = classification_report(y_test_bin, y_pred_bin, target_names=['A', 'B'])
print(report)

# 결과 출력
print("Micro Precision:", precision_micro)
print("Macro Precision:", precision_macro)
print("Weighted Precision:", precision_weighted)

print("Micro Recall:", recall_micro)
print("Macro Recall:", recall_macro)
print("Weighted Recall:", recall_weighted)

print("Micro F1 Score:", f1_micro)
print("Macro F1 Score:", f1_macro)
print("Weighted F1 Score:", f1_weighted)


              precision    recall  f1-score   support

           A       0.96      1.00      0.98       193
           B       0.93      0.97      0.95       174

   micro avg       0.95      0.98      0.97       367
   macro avg       0.95      0.98      0.96       367
weighted avg       0.95      0.98      0.97       367
 samples avg       0.95      0.98      0.95       367

Micro Precision: 0.94750656167979
Macro Precision: 0.9465883977900552
Weighted Precision: 0.947541587005284
Micro Recall: 0.9836512261580381
Macro Recall: 0.9827586206896552
Weighted Recall: 0.9836512261580381
Micro F1 Score: 0.9652406417112299
Macro F1 Score: 0.9643335842024156
Weighted F1 Score: 0.9652579425356487


In [25]:
import pandas as pd
report_df = pd.read_csv(pd.io.common.StringIO(report), sep='\s+')
report_df

ParserError: Error tokenizing data. C error: Expected 5 fields in line 6, saw 6
